In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

%matplotlib inline

In [285]:
train = pd.read_csv('./data/Train_Fyxd0t8.csv', index_col='ID')
test = pd.read_csv('./data/Test_C1XBIYq.csv', index_col='ID')

In [286]:
%run scripts/features.py
%run scripts/eval.py

In [287]:
# impute missing values based on Pesticide_Use_Category

# mean_per_pesticide_use_category_train = train.groupby('Pesticide_Use_Category')['Number_Weeks_Used'].mean()

# def impute_values_train(row):
#     if pd.isnull(row.Number_Weeks_Used):
#         return mean_per_pesticide_use_category_train.ix[row.Pesticide_Use_Category]
#     else:
#         return row.Number_Weeks_Used

# train['Number_Weeks_Used'] = train.apply(impute_values_train, axis=1)

train = train.fillna(0)
test = test.fillna(0)

In [288]:
# mean_per_pesticide_use_category_test = test.groupby('Pesticide_Use_Category')['Number_Weeks_Used'].mean()

# def impute_values_test(row):
#     if pd.isnull(row.Number_Weeks_Used):
#         return mean_per_pesticide_use_category_test.ix[row.Pesticide_Use_Category]
#     else:
#         return row.Number_Weeks_Used

# test['Number_Weeks_Used'] = test.apply(impute_values_test, axis=1)

In [289]:
y = train.Crop_Damage

In [290]:
from sklearn.cross_validation import train_test_split

In [291]:
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.2, random_state=44)

In [292]:
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix

import xgboost as xgb

In [293]:
ft = FeatureTransformer()
# scaler = MinMaxScaler()
# clf = LogisticRegression(C=0.001)
# clf = KNeighborsClassifier()
# clf = RandomForestClassifier()
clf = xgb.XGBClassifier(n_estimators=300, learning_rate=0.08, max_depth=6, subsample=0.8, colsample_bytree=0.8)

pipeline = Pipeline([('ft', ft), ('clf', clf)])

In [294]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('ft', FeatureTransformer()), ('clf', XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.08, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=None, n_estimators=300, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.8))])

In [295]:
# cross validation
scores = eval_models([pipeline], X_train, y_train)

accuracy score: 0.845337
combined score: 0.845337
accuracy score: 0.839148
combined score: 0.839148
accuracy score: 0.845900
combined score: 0.845900
accuracy score: 0.846533
combined score: 0.846533
accuracy score: 0.844704
combined score: 0.844704


In [281]:
print 'Mean Scores %f ' %(np.mean(scores)) 

Mean Scores 0.846111 


In [282]:
predsTest = pipeline.predict(X_test)
print 'Classification report %s ' %(classification_report(y_test, predsTest))

Classification report              precision    recall  f1-score   support

          0       0.86      0.98      0.92     14837
          1       0.51      0.19      0.28      2457
          2       0.24      0.01      0.02       478

avg / total       0.80      0.84      0.81     17772
 


In [283]:
confusion_matrix(y_test, predsTest)

array([[14534,   300,     3],
       [ 1971,   476,    10],
       [  325,   149,     4]])

In [63]:
# fit on the whole dataset
pipeline.fit(train, y)

Pipeline(steps=[('ft', FeatureTransformer()), ('clf', XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.08, max_delta_step=0, max_depth=6,
       min_child_weight=3, missing=None, n_estimators=300, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.8))])

In [64]:
predictions = pipeline.predict(test)

In [65]:
submissions = pd.read_csv('./data/Sample_Submission_Psj3sjG.csv')

In [66]:
submissions['ID'] = test.index.values
submissions['Crop_Damage'] = predictions

In [67]:
submissions.to_csv('./submissions/tenth.csv', index=False)